In [13]:
import numpy as np
from heapq import heappop, heappush
from itertools import count


In [14]:

PUZZLE_DIM = 3
GOAL_STATE = np.arange(1, PUZZLE_DIM**2).tolist() + [0]
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]



In [15]:

def heuristic(state):
    #manhattan distance
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:
                goal_x, goal_y = divmod(value - 1, PUZZLE_DIM)
                distance += abs(x - goal_x) + abs(y - goal_y)
    return distance


def find_zero(state):
    #empty tile
    return tuple(np.argwhere(state == 0)[0])


def available_moves(state):
    #valid moves
    zero_x, zero_y = find_zero(state)
    moves = []
    for dx, dy in DIRECTIONS:
        new_x, new_y = zero_x + dx, zero_y + dy
        if 0 <= new_x < PUZZLE_DIM and 0 <= new_y < PUZZLE_DIM:
            new_state = state.copy()
            new_state[zero_x, zero_y], new_state[new_x, new_y] = (
                new_state[new_x, new_y],
                new_state[zero_x, zero_y],
            )
            moves.append(new_state)
    return moves


In [16]:


def a_star(initial_state, goal_state):
    #A*
    open_set = []
    visited = set()
    counter = count()
    total_evaluated = 0

    heappush(open_set, (heuristic(initial_state), next(counter), initial_state, []))

    while open_set:
        _, _, current_state, path = heappop(open_set)

        if np.array_equal(current_state, goal_state):
            return path + [current_state], total_evaluated

        state_key = tuple(current_state.flatten())
        if state_key in visited:
            continue
        visited.add(state_key)
        total_evaluated += 1

        for move in available_moves(current_state):
            if tuple(move.flatten()) not in visited:
                heappush(
                    open_set,
                    (
                        len(path) + 1 + heuristic(move),
                        next(counter),
                        move,
                        path + [current_state],
                    ),
                )
    return None, total_evaluated




In [17]:

def generate_puzzle(dim, num_moves=50):
    #random puzzle
    state = np.array(GOAL_STATE).reshape((dim, dim))
    for _ in range(num_moves):
        moves = available_moves(state)
        state = moves[np.random.randint(len(moves))]
    return state


In [18]:


if __name__ == "__main__":
    initial_state = generate_puzzle(PUZZLE_DIM, num_moves=20)
    print("Initial state:")
    print(initial_state)

    solution_path, cost = a_star(initial_state, np.array(GOAL_STATE).reshape((PUZZLE_DIM, PUZZLE_DIM)))

    if solution_path:
        print(f" (Quality): {len(solution_path) - 1}")
        print(f" (Cost): {cost}")
        print("\nSolution path:")
        for step, state in enumerate(solution_path):
            print(f"\n{state}\n")
    else:
        print("\nNo solution found.")
        print(f" (Cost): {cost}")

Initial state:
[[2 3 6]
 [1 0 4]
 [7 5 8]]
 (Quality): 8
 (Cost): 10

Solution path:

[[2 3 6]
 [1 0 4]
 [7 5 8]]


[[2 3 6]
 [1 4 0]
 [7 5 8]]


[[2 3 0]
 [1 4 6]
 [7 5 8]]


[[2 0 3]
 [1 4 6]
 [7 5 8]]


[[0 2 3]
 [1 4 6]
 [7 5 8]]


[[1 2 3]
 [0 4 6]
 [7 5 8]]


[[1 2 3]
 [4 0 6]
 [7 5 8]]


[[1 2 3]
 [4 5 6]
 [7 0 8]]


[[1 2 3]
 [4 5 6]
 [7 8 0]]

